In [1]:
from typing import List
import numpy as np
import pandas as pd

In [2]:
#reason we have categorical datatypes:
#some columns consist of a set of distinct values
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)

In [3]:
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [4]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [5]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [6]:
#One storage technique is to use "dimension tables"
#Dimension table contains the distinct values
#primary options are stored as integer keys
values = pd.Series([0, 1, 0, 0] * 2)

In [7]:
dim = pd.Series(['apple', 'orange'])

In [8]:
values #the list of numeric values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [9]:
dim

0     apple
1    orange
dtype: object

In [10]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

In [11]:
#Integer representation is the categorical representation.
#array of distinct values is the categories, dictionary, or levels of the data
#Pandas Categorical extension type
#faster than string data
fruits: List[str] = ['apple', 'orange', 'apple', 'apple'] * 2
fruits

['apple', 'orange', 'apple', 'apple', 'apple', 'orange', 'apple', 'apple']

In [12]:
N = len(fruits)
N

8

In [13]:
rng = np.random.default_rng(seed=12345)
rng

Generator(PCG64) at 0x7FE1B928BD80

In [14]:
df = pd.DataFrame({'fruit': fruits,
                  'basket_id': np.arange(N),
                  'count': rng.integers(3, 15, size=N),
                  'weight': rng.uniform(0, 4, size=N)},
                 columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,11,1.564438
1,1,orange,5,1.331256
2,2,apple,12,2.393235
3,3,apple,6,0.746937
4,4,apple,5,2.691024
5,5,orange,12,3.767211
6,6,apple,10,0.992983
7,7,apple,11,3.795525


In [15]:
df['fruit']

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: object

In [16]:
df['fruit'].dtype

dtype('O')

In [17]:
#convert df['fruit'] to a category
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [18]:
#values for fruit_cat are an instance of pd.Categorical
#we can access the values with the .array attribute
c = fruit_cat.array

In [19]:
type(c)

pandas.core.arrays.categorical.Categorical

In [20]:
#categorical object has categories and codes
c.categories

Index(['apple', 'orange'], dtype='object')

In [21]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [22]:
#the cat accessor (pg. 242 will give more info about the categories and codes methods)
#trick to get mapping between codes and categories
dict(enumerate(c.categories))

{0: 'apple', 1: 'orange'}

In [23]:
#convert a DataFrame column to category by assigning the converted result
df['fruit'] = df['fruit'].astype('category')

In [24]:
df['fruit']

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [25]:
#Create pd.Categorical objects directly from Python sequences
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])

In [26]:
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [27]:
#if we have categorical data from a different sources, we can use the
#from_codes constructor
categories : List = ['foo', 'bar', 'baz']

In [28]:
codes = [0, 1, 2, 0, 0, 1]

In [29]:
my_cats_2 = pd.Categorical.from_codes(codes, categories)

In [30]:
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [31]:
#note: Categorical data doesn't have a specific order unless specified.
ordered_cat = pd.Categorical.from_codes(codes, categories,
                                       ordered=True)
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [32]:
#convert unordered categorical instances to ordered categorical instances with
#.as_ordered
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [33]:
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [34]:
#Note: non-string datatypes also work with categories
#Make random categorical data and use pandas.qcut
rng = np.random.default_rng(seed=12345)

In [35]:
rng

Generator(PCG64) at 0x7FE19885CD60

In [36]:
draws = rng.standard_normal(1000)

In [37]:
draws[:5]

array([-1.42382504,  1.26372846, -0.87066174, -0.25917323, -0.07534331])

In [38]:
#compute a quartile binning and extract some statistics
bins = pd.qcut(draws, 4)

In [39]:
bins

[(-3.121, -0.675], (0.687, 3.211], (-3.121, -0.675], (-0.675, 0.0134], (-0.675, 0.0134], ..., (0.0134, 0.687], (0.0134, 0.687], (-0.675, 0.0134], (0.0134, 0.687], (-0.675, 0.0134]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.121, -0.675] < (-0.675, 0.0134] < (0.0134, 0.687] < (0.687, 3.211]]

In [40]:
#what if the actual sample quartiles are less useful
#than the labels
#we can achieve this with the pd.qcut argument which takes a list of labels
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

In [41]:
bins

['Q1', 'Q4', 'Q1', 'Q2', 'Q2', ..., 'Q3', 'Q3', 'Q2', 'Q3', 'Q2']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [42]:
#bins doesn't contain information about the bin edges, so let's use groupby
#to extract summary statistics
bins = pd.Series(bins, name='quartile')
bins

0      Q1
1      Q4
2      Q1
3      Q2
4      Q2
       ..
995    Q3
996    Q3
997    Q2
998    Q3
999    Q2
Name: quartile, Length: 1000, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [43]:
results = (pd.Series(draws)
          .groupby(bins)
          .agg(['count', 'min', 'max'])
          .reset_index())
results

,quartile,count,min,max
0,Q1,250,-3.119609,-0.678494
1,Q2,250,-0.673305,0.008009
2,Q3,250,0.018753,0.686183
3,Q4,250,0.688282,3.211418


In [44]:
#the quartile column retains the original categorical information, including ordering
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [45]:
#better performance with categoricals
N = 10_000_000

In [46]:
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

In [47]:
labels

0          foo
1          bar
2          baz
3          qux
4          foo
          ... 
9999995    qux
9999996    foo
9999997    bar
9999998    baz
9999999    qux
Length: 10000000, dtype: object

In [48]:
#convert labels to categorical
categories = labels.astype('category')

In [49]:
labels.memory_usage(deep=True)

600000128

In [50]:
categories.memory_usage(deep=True)

10000540

In [51]:
#conversion to category is not free but is a one time cost
%time
_ = labels.astype('category')

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.82 µs


In [52]:
#groupby is a lot faster with category
%timeit labels.value_counts()

324 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [53]:
%timeit categories.value_counts()

17.3 ms ± 60.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [54]:
#categorical methods: special methods for categories
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: object

In [58]:
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [57]:
s.astype(pd.CategoricalDtype()) #same as above

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [59]:
#accessor attibute cat provides access to categorical methods
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [61]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [62]:
#let's say that there's a greater variety of categories
actual_categories : List[str] = ['a', 'b', 'c', 'd', 'e']

In [63]:
actual_categories

['a', 'b', 'c', 'd', 'e']

In [64]:
cat_s2 = cat_s.cat.set_categories(actual_categories)

In [65]:
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

In [66]:
#the data appears unchanged, but the categories will be reflected in operations that use them
cat_s.value_counts()

a    2
b    2
c    2
d    2
dtype: int64

In [67]:
cat_s2.value_counts()

a    2
b    2
c    2
d    2
e    0
dtype: int64

In [68]:
#Categorical data is often used to save memory on larger datasets. 
#Consequently, after we filter data, some categories may no longer be present.
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [70]:
#s.cat.remove_unused_categories() : removes categories that are not found in the data.
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

In [73]:
#let's practice with customer ages
cat_ages_s0 = pd.Series(np.random.randint(22, 32, 100000))
cat_ages_s0.head()

0    29
1    23
2    24
3    29
4    31
dtype: int64

In [75]:
cat_ages_s1 = cat_ages_s0.astype('string').astype('category')
cat_ages_s1

0        29
1        23
2        24
3        29
4        31
         ..
99995    26
99996    27
99997    27
99998    31
99999    23
Length: 100000, dtype: category
Categories (10, string): [22, 23, 24, 25, ..., 28, 29, 30, 31]

In [78]:
cat_ages_s1.cat.add_categories(['33'])

0        29
1        23
2        24
3        29
4        31
         ..
99995    26
99996    27
99997    27
99998    31
99999    23
Length: 100000, dtype: category
Categories (11, object): ['22', '23', '24', '25', ..., '29', '30', '31', '33']

In [80]:
cat_ages_s1.cat.codes

0        7
1        1
2        2
3        7
4        9
        ..
99995    4
99996    5
99997    5
99998    9
99999    1
Length: 100000, dtype: int8

In [84]:
cat_ages_s1.cat.rename_categories(list('abcdefghij'))

0        h
1        b
2        c
3        h
4        j
        ..
99995    e
99996    f
99997    f
99998    j
99999    b
Length: 100000, dtype: category
Categories (10, object): ['a', 'b', 'c', 'd', ..., 'g', 'h', 'i', 'j']

In [85]:
cat_ages_s1.cat.as_ordered()

0        29
1        23
2        24
3        29
4        31
         ..
99995    26
99996    27
99997    27
99998    31
99999    23
Length: 100000, dtype: category
Categories (10, string): [22 < 23 < 24 < 25 ... 28 < 29 < 30 < 31]

In [86]:
cat_ages_s1.cat.as_unordered()

0        29
1        23
2        24
3        29
4        31
         ..
99995    26
99996    27
99997    27
99998    31
99999    23
Length: 100000, dtype: category
Categories (10, string): [22, 23, 24, 25, ..., 28, 29, 30, 31]

In [92]:
reversed_categories = list(cat_ages_s1.cat.categories)[::-1]

In [93]:
#replace each category with a flipped list of categories
cat_ages_s1.cat.rename_categories(reversed_categories)

0        24
1        30
2        29
3        24
4        22
         ..
99995    27
99996    26
99997    26
99998    22
99999    30
Length: 100000, dtype: category
Categories (10, object): ['31', '30', '29', '28', ..., '25', '24', '23', '22']

In [94]:
cat_ages_s1.value_counts()

26    10192
30    10144
25    10072
29    10053
31    10014
24     9975
28     9943
22     9927
23     9869
27     9811
dtype: int64

In [95]:
#create dummy variables for modelling using categorical data
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

In [97]:
pd.get_dummies(cat_s) #each column where an item is present is a 1

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1
